# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [6]:
df.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [ ]:
!pip install langchain_community

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [9]:
prompt = ChatPromptTemplate.from_template( 'Give a list 3 sumarised reviews and general sentiment evaluation {data}')
 


In [10]:

chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product = 'Luxury Air Mattress'
chain.run(product)

'1. "This luxury air mattress is absolutely phenomenal. It\'s like sleeping on a cloud, so comfortable and supportive. Worth every penny!" - Positive sentiment\n\n2. "I purchased this luxury air mattress and was really disappointed. It didn\'t hold air well and I woke up on the floor multiple times. Not worth the investment." - Negative sentiment\n\n3. "I was skeptical about buying a luxury air mattress, but this one exceeded my expectations. It\'s durable, easy to inflate/deflate, and provides an excellent night\'s sleep. I highly recommend it." - Positive sentiment'

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    ' Give a the top 5 product sentsentiments from {data}'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [14]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'Give a table of the top 5 product sentiments and their respective counts {data}'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [16]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
1. "This luxury air mattress exceeded all of my expectations - it's incredibly comfortable and durable."
2. "I've never slept better than on this luxurious air mattress - it's like sleeping on a cloud."
3. "The quality and craftsmanship of this luxury air mattress is unmatched - worth every penny."
4. "I didn't know an air mattress could be so luxurious until I tried this one - it's truly a game changer."
5. "I highly recommend investing in this luxury air mattress for the ultimate in comfort and relaxation."
| Product Sentiment                                                                                 | Count |
|----------------------------------------------------------------------------------------------------|-------|
| This luxury air mattress exceeded all of my expectations - it's incredibly comfortable and durable. | 1     |
| I've never slept better than on this luxurious air mattress - it's like sleeping on a cloud.         |

"| Product Sentiment                                                                                 | Count |\n|----------------------------------------------------------------------------------------------------|-------|\n| This luxury air mattress exceeded all of my expectations - it's incredibly comfortable and durable. | 1     |\n| I've never slept better than on this luxurious air mattress - it's like sleeping on a cloud.         | 1     |\n| The quality and craftsmanship of this luxury air mattress is unmatched - worth every penny.         | 1     |\n| I didn't know an air mattress could be so luxurious until I tried this one - it's truly a game changer. | 1     |\n| I highly recommend investing in this luxury air mattress for the ultimate in comfort and relaxation. | 1     |"

**Repeat the above twice for different products**

## SequentialChain

In [107]:
pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\users\sadiq\onedrive\documents\anaconda3\envs\new_hack_lib\lib\site-packages
Requires: async-timeout, langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [61]:
from langchain.chains import SequentialChain

In [93]:
# Define individual components
#prompt1 = PromptTemplate(template="Translate English to French: {text}", input_variables=["text"])
#prompt2 = PromptTemplate(template="Translate French to Spanish: {french_text}", input_variables=["french_text"])





llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    " {data}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='English'
                    )


In [100]:
chain_one = PromptTemplate(
    input_variables=["Review"],
    template="Rewrite the following review in clear English:\n\n{Review}"
)
chain_one = LLMChain(llm=llm, prompt=p1, output_key="English")

In [101]:
# 2) Summarize the English review
chain_two = PromptTemplate(
    input_variables=["English"],
    template="Give a table of the top 5 product sentiments and their respective counts {data}:\n\n{English}"
)
chain_two = LLMChain(llm=llm, prompt=p2, output_key="Summary")

In [102]:
# 3) Draft a follow-up message
chain_three = PromptTemplate(
    input_variables=["English", "Summary"],
    template=("Write a polite follow-up message to the reviewer, "
              "using the English text and its summary.\n\n"
              "English:\n{English}\n\nSummary:\n{Summary}")
)
chain_three = LLMChain(llm=llm, prompt=p3, output_key="Followup")

In [103]:
# 4) Translate the English review to French
chain_four = PromptTemplate(
    input_variables=["English"],
    template="Translate to French:\n\n{English}"
)
chain_four = LLMChain(llm=llm, prompt=p4, output_key="French")

In [104]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],                   # <- name used when calling the chain
    output_variables=["English", "Summary", "Followup", "French"],  # <- unique, real keys
    verbose=True
)

In [105]:
# Run
result = overall_chain({"Review": "orig review text here"})
print(result["English"])
print(result["Summary"])
print(result["Followup"])
print(result["French"])



> Entering new SequentialChain chain...

> Finished chain.
"this restaurant is the bomb. the food is off the hook and the service is on point. definitely check it out if you want a great meal." 

Revised: "This restaurant is excellent. The food is delicious and the service is top-notch. I highly recommend checking it out for a great meal."
This review praises the restaurant for its delicious food and excellent service, recommending it for a great meal.
Dear reviewer,

Thank you for your kind words about our restaurant. We are delighted to hear that you enjoyed the delicious food and top-notch service. We appreciate your recommendation and hope to welcome you back soon for another great meal.

Best regards, [Your Name]
Ce restaurant est excellent. La nourriture est délicieuse et le service est de qualité. Je recommande vivement de venir pour un excellent repas.


In [106]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [108]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English': "I find the taste poor. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'Summary': 'The reviewer finds the taste of this product to be poor and notes that the foam does not hold up well. They suspect that they may have received an old batch or a counterfeit product, as they have had better experiences with the same item purchased in stores.',
 'Followup': "Dear Reviewer,\n\nThank you for providing feedback on your recent purchase. I'm sorry to hear that you found the taste of the product to be poor and that the foam didn't hold up well. We take the quality of our products very seriously and would like to investigate this further. It's possible that you received an old batch or a counterfeit item, and we want to make thi

**Repeat the above twice for different products or reviews**

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**